In [1]:
# Changed code relative to others:
# encoding no longer use cos sin (no noise)
# dump log now dump both pickle and txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from scipy import stats
from tqdm import tqdm_notebook

import Config
from Config import *
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [4]:
# train data once. This is without retraining so beware!
def onepass_train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = hdc.fit_once(traindata, trainlabels, param)
    test_acc = hdc.test(testdata, testlabels)
    print("Train: %f \t \t Test: %f"%(train_acc, test_acc))
    return train_acc, test_acc

In [5]:
# train data. with retrain
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_accs = []
    test_accs = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_accs.append(hdc.fit(traindata, trainlabels, param))
        test_accs.append(hdc.test(testdata, testlabels, param["kernel"]))
        if len(train_accs) % 5 == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
        if train_accs[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
            break
    return np.asarray(train_accs), np.asarray(test_accs)

In [6]:
# train data once via batch
def train_batch(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config, batch = 50):
    train_accs = []
    for i in range(len(traindata)//batch):
        train_accs.append(hdc.fit_sep(traindata[i*batch: (i+1)*batch], trainlabels[i*batch: (i+1)*batch], param))
    train_acc = np.mean(np.asarray(train_accs))
    test_acc = hdc.test(testdata, testlabels, param["kernel"])
    #print("Train: %f \t \t Test: %f"%(train_acc, test_acc))
    return train_acc, test_acc

In [7]:
# train data via batch, with retrain
def batched_train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config, batch = 50):
    train_accs = []
    test_accs = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc, test_acc = train_batch(hdc, traindata, trainlabels, testdata, testlabels, param, batch)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        if len(train_accs) % 5 == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
        if train_accs[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
            break
    return np.asarray(train_accs), np.asarray(test_accs)

In [8]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "w")
    file.write("Max train: %.2f \tMax test: %.2f \n"%(train_acc*100, test_acc*100))
    file.write(str(param))
    file.close()

In [9]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()


Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [10]:
#Data shuffling 
shuf_train = np.random.permutation(len(traindata))
traindata = traindata[shuf_train]
trainlabels = trainlabels[shuf_train]

shuf_test = np.random.permutation(len(testdata))
testdata = testdata[shuf_test]
testlabels = testlabels[shuf_test]

In [11]:
traindata = traindata[:10000]
testdata = testdata[:5000]

In [12]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 2000, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'checkpoints': False, 'kernel': <Kernel_T.DOT: 0>, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 3, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 80, 'nFeatures': 784, 'nClasses': 10}


In [13]:
################# VANILLA #################
hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

Generating vanilla HD basis of shape... 


6310


(2000, 784)
Encoding time: 0.10472846031188965 


In [14]:
# Retrieve info upto basis generator, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [15]:
hde = HDE.HD_encoder(basis)

trainencoded = hde.encodeData(traindata)
#HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
#HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (10000, 784)


Time spent: 4 sec
Encoding data of shape (5000, 784)


Time spent: 2 sec


In [16]:
# Retrieve info upto encoder, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)
#trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
#testencoded, testlabels = HDE.loadEncoded("encoded_%d_test.pkl"%bid)

# Onepass - train

In [17]:
# Should have 90%
one_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = onepass_train(one_hdc, trainencoded, trainlabels, testencoded, testlabels)

Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.DOT: 0>)] 


Train: -1.000000 	 	 Test: 0.869147


In [18]:
accs = dict()
for k in Kernel_T:
    accs[k]= one_hdc.test(testencoded, testlabels, k)

In [19]:
for k in Kernel_T:
    print(k, accs[k])

Kernel_T.DOT 0.851
Kernel_T.COS 0.851
Kernel_T.PL1 0.7984
Kernel_T.PL3 0.8524
Kernel_T.PL4 0.852
Kernel_T.BT1 0.8346
Kernel_T.BT3 0.8534
Kernel_T.BT4 0.8538


# Onepass - train - automation

In [18]:
def onepass_automation(param):
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    # Should have 90%
    one_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
    train_acc, test_acc = onepass_train(one_hdc, trainencoded, trainlabels, testencoded, testlabels)
    
    accs = dict()
    for k in Kernel_T:
        accs[k]= one_hdc.test(testencoded, testlabels, k)
        print(k, accs[k])
    return accs

In [19]:
accs_D = dict()
for D in [500, 1000, 2000, 3000, 4000, 5000]:
    param["D"] = D
    accs_D[D] = onepass_automation(param)

Generating vanilla HD basis of shape... 

(500, 617)
Encoding time: 0.037902116775512695 
Encoding data of shape (6238, 617)


Time spent: 0 sec
Encoding data of shape (1559, 617)


Time spent: 0 sec
Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.DOT: 0>)] 


Train: -1.000000 	 	 Test: 0.865298
Kernel_T.DOT 0.8652982681205901
Kernel_T.COS 0.8652982681205901
Kernel_T.PL1 0.742142398973701
Kernel_T.PL3 0.8441308531109686
Kernel_T.PL4 0.8524695317511225
Kernel_T.BT1 0.7761385503527902
Kernel_T.BT3 0.8550352790250161
Kernel_T.BT4 0.8543938422065427


...

Generating vanilla HD basis of shape... 

(5000, 617)
Encoding time: 0.17552947998046875 
Encoding data of shape (6238, 617)


Time spent: 7 sec
Encoding data of shape (1559, 617)


Time spent: 1 sec
Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.DOT: 0>)] 


Train: -1.000000 	 	 Test: 0.878768
Kernel_T.DOT 0.8787684413085312
Kernel_T.COS 0.8787684413085312
Kernel_T.PL1 0.8588838999358563
Kernel_T.PL3 0.8723540731237973
Kernel_T.PL4 0.8717126363053239
Kernel_T.BT1 0.8524695317511225
Kernel_T.BT3 0.8742783835792175
Kernel_T.BT4 0.8723540731237973


In [20]:
for D in accs_D.keys():
    print(D)
    for k in accs_D[D]:
        print(accs_D[D][k])

500
0.8652982681205901
0.8652982681205901
0.742142398973701
0.8441308531109686
0.8524695317511225
0.7761385503527902
0.8550352790250161
0.8543938422065427
1000
0.8774855676715844
0.8774855676715844
0.8120590121872996
0.8691468890314304
0.8659397049390635
0.8364336112892881
0.8736369467607441
0.8736369467607441
2000
0.8691468890314304
0.8691468890314304
0.8402822322001283
0.8659397049390635
0.8608082103912764
0.8377164849262347
0.8697883258499037
0.8672225785760103
3000
0.876844130853111
0.876844130853111
0.844772289929442
0.8762026940346376
0.8762026940346376
0.8678640153944837
0.8742783835792175
0.876844130853111
4000
0.8800513149454778
0.8800513149454778
0.8640153944836434
0.8787684413085312
0.8774855676715844
0.8774855676715844
0.876844130853111
0.876844130853111
5000
0.8787684413085312
0.8787684413085312
0.8588838999358563
0.8723540731237973
0.8717126363053239
0.8524695317511225
0.8742783835792175
0.8723540731237973


# Comparing test results - retrain

In [32]:
hdcs = dict()
accs = dict()

In [18]:

param["kernel"] = k = Kernel_T.DOT
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.DOT: 0>)] 


Train: 0.676200 	 	 Test: 0.869600
Train: 0.925100 	 	 Test: 0.935400
Train: 0.962800 	 	 Test: 0.887600
Train: 0.959800 	 	 Test: 0.950200
Train: 0.982100 	 	 Test: 0.943400
Train: 0.985800 	 	 Test: 0.905000
Train: 0.992400 	 	 Test: 0.937200
Train: 0.989700 	 	 Test: 0.952000
Train: 0.958600 	 	 Test: 0.951800
Train: 0.995000 	 	 Test: 0.943400
Train: 0.997700 	 	 Test: 0.955400
Train: 0.998400 	 	 Test: 0.955200
Train: 0.997900 	 	 Test: 0.955400
Train: 0.998700 	 	 Test: 0.939400
Train: 0.998500 	 	 Test: 0.934800
Train: 0.997800 	 	 Test: 0.956800
Train: 0.999200 	 	 Test: 0.955800
Train: 0.999000 	 	 Test: 0.955800
Train: 0.998800 	 	 Test: 0.956000
Train: 1.000000 	 	 Test: 0.955400
Train: 1.000000 	 	 Test: 0.955400


In [19]:

param["kernel"] = k = Kernel_T.PL1
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL1: 2>)] 


Train: 0.598100 	 	 Test: 0.853000
Train: 0.844400 	 	 Test: 0.662000
Train: 0.849800 	 	 Test: 0.830400
Train: 0.879000 	 	 Test: 0.913400
Train: 0.869700 	 	 Test: 0.876000
Train: 0.854600 	 	 Test: 0.756600
Train: 0.901500 	 	 Test: 0.896800
Train: 0.852600 	 	 Test: 0.888400
Train: 0.904900 	 	 Test: 0.883400
Train: 0.852400 	 	 Test: 0.911000



In [20]:

param["kernel"] = k = Kernel_T.PL3
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL3: 3>)] 


Train: 0.803300 	 	 Test: 0.872600
Train: 0.962400 	 	 Test: 0.884600
Train: 0.978500 	 	 Test: 0.940000
Train: 0.973300 	 	 Test: 0.944600
Train: 0.992100 	 	 Test: 0.949000
Train: 0.993200 	 	 Test: 0.948400
Train: 0.996800 	 	 Test: 0.948400
Train: 0.997700 	 	 Test: 0.951600
Train: 0.998300 	 	 Test: 0.948800
Train: 0.995900 	 	 Test: 0.944200



In [21]:

param["kernel"] = k = Kernel_T.PL4
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL4: 4>)] 


Train: 0.790800 	 	 Test: 0.888200
Train: 0.969200 	 	 Test: 0.934200
Train: 0.985700 	 	 Test: 0.935400
Train: 0.990000 	 	 Test: 0.947800
Train: 0.992500 	 	 Test: 0.946600
Train: 0.989000 	 	 Test: 0.942200
Train: 0.998300 	 	 Test: 0.942200
Train: 0.999400 	 	 Test: 0.949600
Train: 0.998200 	 	 Test: 0.951800
Train: 0.986300 	 	 Test: 0.949000



In [ ]:
param["kernel"] = k = Kernel_T.BT1
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)

# preprocess data every epoch is not every time economic
trainprocessed = hdc.preprocess_data(trainencoded, k)
testprocessed = hdc.preprocess_data(testencoded, k)

train_acc, test_acc = batched_train(hdc, trainprocessed, trainlabels, testprocessed, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.BT1: 5>)] 


Train: 0.795500 	 	 Test: 0.847000
Train: 0.918200 	 	 Test: 0.897400
Train: 0.935900 	 	 Test: 0.902400
Train: 0.936700 	 	 Test: 0.724400
Train: 0.938300 	 	 Test: 0.897000
Train: 0.937400 	 	 Test: 0.893600
Train: 0.947300 	 	 Test: 0.904600
Train: 0.949300 	 	 Test: 0.899600
Train: 0.946700 	 	 Test: 0.889400
Train: 0.947900 	 	 Test: 0.886600
Train: 0.937000 	 	 Test: 0.896400


In [31]:
param["kernel"] = k = Kernel_T.BT3
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


AssertionError: 

In [ ]:
param["kernel"] = k = Kernel_T.BT4
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

# Retrained - automation

In [17]:
def retrain_type(param, kernel_t, trainencoded, trainlabels, testencoded, testlabels):
    
    param["kernel"] = k = kernel_t
    hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)

    # preprocess data every epoch is not every time economic
    trainprocessed = hdc.preprocess_data(trainencoded, k)
    testprocessed = hdc.preprocess_data(testencoded, k)

    train_acc, test_acc = batched_train(hdc, trainprocessed, trainlabels, testprocessed, testlabels, param)
    
    return max(test_acc), hdc.test(testencoded, testlabels, k)

In [19]:
def retrain_automation(param):
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    accs = dict()
    for k in [Kernel_T.BT1, Kernel_T.BT3, Kernel_T.BT4]: #Kernel_T: #
        accs[k] = retrain_type(param, k, trainencoded, trainlabels, testencoded, testlabels)
        print(k, accs[k])
    return accs

In [ ]:
accs_D = dict()
for D in [500, 1000, 2000]:
    param["D"] = D
    accs_D[D] = retrain_automation(param)

Generating vanilla HD basis of shape... 

(500, 784)
Encoding time: 0.04891347885131836 
Encoding data of shape (10000, 784)


Time spent: 1 sec
Encoding data of shape (5000, 784)


Time spent: 0 sec
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.BT1: 5>)] 


...

Time spent: 0 sec


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.BT1: 5>)] 


Train: 0.734400 	 	 Test: 0.836200
Train: 0.914300 	 	 Test: 0.863200
Train: 0.930000 	 	 Test: 0.875600
Train: 0.936900 	 	 Test: 0.871000
Train: 0.938900 	 	 Test: 0.883200
Train: 0.941000 	 	 Test: 0.882400
Train: 0.943400 	 	 Test: 0.875200
Train: 0.946400 	 	 Test: 0.891600
Train: 0.945700 	 	 Test: 0.881400
Train: 0.950100 	 	 Test: 0.886000
Train: 0.947700 	 	 Test: 0.869000
Train: 0.947400 	 	 Test: 0.882400
Train: 0.946200 	 	 Test: 0.860400
Train: 0.946800 	 	 Test: 0.863800
Train: 0.948000 	 	 Test: 0.876600
Train: 0.946400 	 	 Test: 0.884800

Kernel_T.BT1 (0.8928, 0.8874)


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.BT3: 6>)] 


Train: 0.840500 	 	 Test: 0.901800
Train: 0.975700 	 	 Test: 0.918400
Train: 0.990000 	 	 Test: 0.925800


In [ ]:
for D in accs_D.keys():
    print(D)
    for k in accs_D[D]:
        x, y = accs_D[D][k]
        print(y)